In [2]:
%matplotlib qt

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import mpl_interactions as pli 

In [4]:
image_1st = np.load('projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_1st_Layer.npy')
image_2nd = np.load('projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_2nd_Layer.npy')
flood_1st = np.load('projections/2024-01-20_16h-55m-58s_080kVp_30mAs_0915_images_Bin2x8LowGain_1st_Layer.npy')
flood_2nd = np.load('projections/2024-01-20_16h-55m-58s_080kVp_30mAs_0915_images_Bin2x8LowGain_2nd_Layer.npy')

In [5]:
plt.figure()
cc = pli.hyperslicer(-np.log(image_2nd/flood_2nd),vmin=0,vmax=1,controls=cont)

libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open radeonsi: /usr/lib/dri/radeonsi_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: radeonsi
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


NameError: name 'cont' is not defined

In [5]:
projs_1st = -np.log(image_1st/flood_1st)
projs_2nd = -np.log(image_2nd/flood_2nd)

In [6]:
header_1st = np.load('projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_1st_Layer_headers.npy',allow_pickle=True)
header_2nd = np.load('projections/2024-01-20_16h-42m-25s_080kVp_30mAs_0915_images_Bin2x8LowGain_2nd_Layer_headers.npy',allow_pickle=True)

In [16]:
# Do a pretty print of the dictionaries header_1st and header_2nd
from pprint import pprint
pprint(header_1st[0])

{b'AcquisitionId': (13, 0, 852867482),
 b'AcquisitionModeId': (17, 2, b'Bin2x8LowGain'),
 b'AcquisitionNotes': (16,
                       2,
                       b'Mode Type : Continuous\r\nOffset Corr. : true\r\nGai'
                       b'n Corr. : true\r\nPixel Corr. : true\r\nRegistration'
                       b' Corr. : false\r\nAveraging : false\r\nIntegration :'
                       b' false\r\nReset Frames : 10\r\nAEC : False'),
 b'AcquisitionSystem': (17, 0, 1),
 b'AcquisitionSystemVersion': (24, 2, b'3.0.117.110'),
 b'BeamState': (9, 0, 3),
 b'CalibrationSetId': (16, 2, b'DefaultCalibrationSetId'),
 b'ControlPoint': (12, 1, 7.989130434782607e-05),
 b'CouchLat': (8, 1, 100.9652407628742),
 b'CouchLng': (8, 1, 58.57526546210167),
 b'CouchPit': (8, 1, -0.0014698505110898458),
 b'CouchRol': (8, 1, -0.000525713009031134),
 b'CouchRtn': (8, 1, 180.0234375),
 b'CouchVrt': (8, 1, 112.8536674416585),
 b'DataGain': (8, 1, 3.4028234663852886e+38),
 b'DataRescaleType': (15, 0, 2

In [7]:
# append the gantry rotation angle from each header to the list angles
angles = []
angles2 = []
for i in range(len(header_1st)):
    # print(header_1st[i].keys())
    angles2.append(header_2nd[i][b'GantryRtn'][2])
    angles.append(header_1st[i][b'GantryRtn'][2])

In [8]:
import tigre

In [9]:
geo = tigre.geometry_default()

In [10]:
geo.DSD = 1500
geo.DSO = 1000
geo.nDetector = np.array(projs_1st[0].shape)
geo.dDetector = np.array([1.12,0.28])
geo.sDetector = geo.nDetector*geo.dDetector


In [17]:
print(geo)

TIGRE parameters
-----
Geometry parameters
Distance from source to detector (DSD) = 1500 mm
Distance from source to origin (DSO)= 1000 mm
-----
Detector parameters
Number of pixels (nDetector) = [ 268 1424]
Size of each pixel (dDetector) = [1.12 0.28] mm
Total size of the detector (sDetector) = [300.16 398.72] mm
-----
Image parameters
Number of voxels (nVoxel) = [512 512 512]
Total size of the image (sVoxel) = [256 256 256] mm
Size of each voxel (dVoxel) = [0.5 0.5 0.5] mm
-----
Offset correction parameters
Offset of image from origin (offOrigin) = [0 0 0] mm
Offset of detector (offDetector) = [0 0] mm
-----
Auxillary parameters
Samples per pixel of forward projection (accuracy) = 0.5


In [31]:
img1_exp = tigre.algorithms.fdk(projs_1st,geo,angles=-np.deg2rad(angles),verbose=True,filter='hamming')

In [32]:
img2_exp = tigre.algorithms.fdk(projs_2nd,geo,angles=-np.deg2rad(angles2),verbose=True,filter='hamming')

In [49]:
fig = plt.figure()
plt.subplot(131)
cc = pli.hyperslicer(img1_exp[412:],cmap='bone',vmin=0.005,vmax=0.011)
plt.title('1st Layer Experimental')
plt.axis('off')
plt.subplot(132)
c2 = pli.hyperslicer(img2_exp[412:],controls=cc,cmap='bone',vmin=0.005,vmax=0.010)
plt.title('2nd Layer Experimental')
plt.axis('off')
plt.subplot(133)
# c3 = pli.hyperslicer(img2_exp/img1_exp,controls=cc,cmap='bwr',vmin=0.9,vmax=1.1)
c3 = pli.hyperslicer((100*(img2_exp-img1_exp)/np.max(img2_exp))[412:],controls=cc,cmap='bwr',vmin=-5,vmax=5)
plt.title('Relative difference (%)')
plt.axis('off')
plt.colorbar()
plt.tight_layout()

[2024-01-31 15:19:50,634] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7fe30fc038d0>


In [23]:
anim = cc.save_animation("xim_readouts.gif", fig, "axis0", interval=80)

In [19]:
phantom_first = np.load('first_layer_360.pkl', allow_pickle=True)
phantom_second = np.load('second_layer_360.pkl', allow_pickle=True)

In [20]:
img1 = phantom_first.img
img2 = phantom_second.img

In [21]:
# delete phantom_first and phantom_second from the memory
del phantom_first
del phantom_second

In [35]:
fig = plt.figure()
plt.subplot(231)
cc = pli.hyperslicer(img1,cmap='bone',vmin=0,vmax=0.02)
plt.title('1st Layer Experimental')
plt.axis('off')
plt.subplot(232)
c2 = pli.hyperslicer(img2,controls=cc,cmap='bone',vmin=0,vmax=0.02)
plt.title('2nd Layer Experimental')
plt.axis('off')
plt.subplot(233)
c3 = pli.hyperslicer(100*(img2-img1)/np.max(img2),controls=cc,cmap='bwr',vmin=-5,vmax=5)
plt.title('Relative difference (%)')
plt.axis('off')
plt.colorbar()
# plt.tight_layout()

# Plot the experimental images
plt.subplot(234)
cc2 = pli.hyperslicer(img1_exp,cmap='bone',vmin=0.002,vmax=0.006)
plt.title('1st Layer Experimental')
plt.axis('off')
plt.subplot(235)
c2 = pli.hyperslicer(img2_exp,controls=cc2,cmap='bone',vmin=0.002,vmax=0.006)
plt.title('2nd Layer Experimental')
plt.axis('off')
plt.subplot(236)
c3 = pli.hyperslicer(100*(img2_exp-img1_exp)/np.max(img2_exp),controls=cc2,cmap='bwr',vmin=-5,vmax=5)
plt.title('Relative difference (%)')
plt.axis('off')
plt.colorbar()
plt.tight_layout()

[2024-01-31 14:44:41,396] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7fe3236ace50>
[2024-01-31 14:44:41,646] {colorbar.py:859} DEBUG - locator: <matplotlib.ticker.AutoLocator object at 0x7fe2e7b86610>
